<script>
    var code_show=true; //true -> hide code at first

    function code_toggle() {
        $('div.prompt').hide(); // always hide prompt

        if (code_show){
            $('div.input').hide();
        } else {
            $('div.input').show();
        }
        code_show = !code_show
    }
    $( document ).ready(code_toggle);
</script>

<center>
<h1>Workshop 1b: Advanced UNIX/Bash</h1><br><br>
<i><big> File operations, pipes and subshells.</big><br><br>
Arthur Gilly (ag15@sanger.ac.uk), Kostas Hatzikotoulas (kh9@sanger.ac.uk)</i>
</center>

**Topics covered in this tutorial**:

* Text processing tools: grep, sed, cut, awk, join, tr and paste.
* Simple variable handling.
* Loops.
* Conditionals.
* Pipes, backticks and brackets.


## Step 1 : Downloading resources from the internet

It is easy to access the internet via the command line. For this we use the `wget` tool. We want to  download the complete [GWAS catalog](https://www.ebi.ac.uk/gwas/) maintained by [EBI](https://www.ebi.ac.uk/) that contains all the published genome-wide associations to date. This collection by 2017.05.12 contains 29,196 SNPs and 33,898 associations publised in 2,882 research articles. You can perform simple queries on the database using the GWAS catalog website, but you can also download it so you can build you own database. 


<div class="alert alert-success"><b>Question 0:</b>  Go to the GWAS catalog website and find the download link. Create a directory in your home named `GWAS_catalog` and download it to a file named `GWAS_catalog_2017.05.12`. How large is the file you just downloaded?
</div>


<button type="button" class="btn btn-primary" onClick="code_toggle()">Click here to show/hide answer</button>



In [ ]:
%%bash

# Goes to your home
cd

# Creates directory and enters it
mkdir GWAS_catalog && cd GWAS_catalog

# After the URL to the dataset, we specify the output file name.
wget https://www.ebi.ac.uk/gwas/api/search/downloads/full -O GWAS_catalog_2017.05.12

# Size
ls -lh GWAS_catalog_2017.05.12

<div class="alert alert-info"> The `&&` operator allows you to execute several commands one after the other, but only if previous ones succeed. For example `command1 && command2 && command3` will execute `command2` only if `command1` succeeded, and `command3` only if both `command1` and `command2` succeeded.
</div>

`wget` is an extremely powerful bash tool, and has a wide range of amazing features (like resuming interrupted download) that we can't cover here. As it has been mentioned in the previous workshop, you can read the comprehensive manual using `man wget`. This also apply to the rest of the tools we are covering in this session.

## Text processing bash tools

###  Step 2: Counting lines, first $n$ and last $n$ lines of a file


<div class="alert alert-success"><b>Question 1:</b>  Using `wc`, check the number of lines in the GWAS catalog. Using `head` and `tail`, display the first 10 lines and the last 5 lines of the file.
</div>


<button type="button" class="btn btn-primary" onClick="code_toggle()">Click here to show/hide answer</button>


In [ ]:
%%bash

# How many lines does the file have?
wc -l GWAS_catalog_2017.05.12

# First 10 rows
head GWAS_catalog_2017.05.12

# Last 5 rows
tail -5 GWAS_catalog_2017.05.12
tail -n 5 GWAS_catalog_2017.05.12


From the first 10 rows we can see how the file is structured: 
* the first line is the header describing the content of each column (a more comprehensive explanation can be found on the [gwas website](https://www.ebi.ac.uk/gwas/docs/fileheaders))
* then each line is an individual association between a genetic variant and an observed phenotype.

### Step 3 : Using pipes

Right now, all the commands we ran were printing their output to the screen (like `wc`) or to a file (`wget -O`). But we can also make the output go to another bash command. We do this using the "pipe" character (`|`). For example, `command1 | command2` will pass the output of `command1` to the input of `command2`. 


<div class="alert alert-success"><b>Question 2:</b>  Using `head` piped to `tail`, print **only the 300th line** of the GWAS catalog. 
<br><br>
`cat` prints the output of one or more files to the screen. Using `cat` and `wc`, print the number of lines in the GWAS catalog.
</div>


<button type="button" class="btn btn-primary" onClick="code_toggle()">Click here to show/hide answer</button>


In [4]:
%%bash

head -300 GWAS_catalog_2017.05.12 | tail -1

cat GWAS_catalog_2017.05.12 | wc -l

2008-06-22	18439548	Ridker PM	2008-04-24	Am J Hum Genet	www.ncbi.nlm.nih.gov/pubmed/18439548	Loci related to metabolic-syndrome pathways including LEPR,HNF1A, IL6R, and GCKR associate with plasma C-reactive protein: the Women's Genome Health Study.	C-reactive protein	6,345 European ancestry female individuals	NA	12q24.31	12	120987058	HNF1A	HNF1A			6927			rs7310409-A	rs7310409	0	7310409	intron_variant	0	NR	7E-17	16.154901959985743		0.15	[NR] mg/dl decrease	Illumina [336108]	N
39690


### Step 3 : redirections

Instead of ouputting the result of a command to another command, we can also tell UNIX to write the output to a file. This is not so useful when you have 1 command, but when you have several piped commands (pipeline) it is used to write the end result to a file. There are 2 types of redirection:

* **overwrite** which is a single "greater than". `command1 > file` will overwrite `file` if it exists, otherwise create it and fill it with the output of `command1`.
* **append**, which is a double "greater than". `command1 >> file` will add the output of `command1` to `file`, creating it if it does not exist.


<div class="alert alert-success"><b>Question 3:</b>  Using `head`, `tail` and pipes, create a file named `300th_line.txt` that contains **only the header and the 300th line**, and display it using `cat`.

</div>


<button type="button" class="btn btn-primary" onClick="code_toggle()">Click here to show/hide answer</button>


In [7]:
%%bash

## First the header
head -1 GWAS_catalog_2017.05.12 > 300th_line.txt

## Then the 300th line
head -300 GWAS_catalog_2017.05.12 | tail -1 >> 300th_line.txt

cat 300th_line.txt

<div class="alert alert-success"><b>Question 4:</b> The file you have created contains both spaces and tab characters as separators, which makes it a bit messy to display. The `column` command can add artificial spaces to a pipe, so that it looks a bit more like Excel. Pipe `cat` into `column -s$'\t' -t`, and then into `less`. By default, `less` folds long lines to make them fit the screen. We have very long lines so we want to display them laterally, without breaking them. Can you find the option in the `less` manual that does that? The 300th line of the GWAS catalog tells us about a mutation in a gene. Can you find the chromosome, position and gene for that variant?

</div>


<button type="button" class="btn btn-primary" onClick="code_toggle()">Click here to show/hide answer</button>


In [ ]:
%%bash

cat 300th_line.txt | column -s$'\t' -t | less -S

# chromosome 12     position 120987058  gene HNF1A

### Searching files

Very often, we want to search the contents of files. This is done using a very powerful command, `grep`.

```bash

grep [OPTIONS] [PATTERN] [FILE]

# Example:

grep Volos cities.txt
```

the last example looks for every line that contains "Volos" in the file `cities.txt` and displays the corresponding row. If your pattern contains a space, enclose it within simple or double quotes (e.g. `"Volos, Thessalia"` or `'Volos, Thessalia'`)

<div class="alert alert-info"> It is possible to search for several patterns at a time, not just one. If you are interested in only a few patterns, you can add them to the command line, separating them by `-e`. For example, ` grep -e Volos -e Thessaloniki cities.txt`. If you have many patterns that you want to search for, you can use `grep -f patterns.txt file.txt` where `patterns.txt` contains all the patterns you are interested in , 1 per line.
</div>
<br>

<div class="alert alert-info"> Interesting parameters for `grep` are `-w`, `-v`, `-i`, `-n`, `-A`, `-B`, `-C`, `-l` and `-L`. Check them out and play around with them if you have time!
</div>

<div class="alert alert-success"><b>Question 5:</b> Use `grep` to search for other variants in the gene you found above. Using a pipe and `wc`, determine how many lines from the GWAS ctalog mention it. How about the variant you found? Are there other lines that mention this variant?

</div>

<div class="alert alert-success"><b>Question 6:</b> Before, we used 2 steps to create a file containing the 300th line. Now with `grep`, we can do everything in one line. Find something unique to the header and our 300th line, and create a file called `300th_line2.txt` that contains them (you might need several grep commands piped together). Use the `diff` command to check that the files are identical.
</div>

<button type="button" class="btn btn-primary" onClick="code_toggle()">Click here to show/hide answers</button>


In [ ]:
%%bash
## Question 5
grep HNF1A GWAS_catalog_2017.05.12
grep HNF1A GWAS_catalog_2017.05.12 | wc -l

grep 120987058 GWAS_catalog_2017.05.12 | wc -l

In [ ]:
%%bash
## Question 6

# There is no single right answer here, you might have selected other search criteria that give the same result.
grep -e DOWNSTREAM_GENE_DISTANCE -e 120987058 GWAS_catalog_2017.05.12| grep -e 2008-06-22 -e DOWNSTREAM_GENE_DISTANCE > 300th_line2.txt

# The diff output is empty: our files are identical!
diff 300th_line.txt 300th_line2.txt

### Step 4: Playing with columns: cut and paste

As we now know, our file contains many columns, but we are interested in only a couple of them. `cut` allows to extract specific columns from a file, whereas `paste` appends several columns together. The delimiter (space, comma, tab) that separates your fields is specified with the `-d` argument (as they are special characters they will need to be quoted with `'`).

#### Example
The below command extracts columns 1 to 4, 6, and 8 to 10 of `file.txt` and prints the output to the screen.
```bash
cut -f1-4,6,8-10 file.txt
```

<div class="alert alert-success"><b>Question 7:</b> Extract the associated disease/trait and the SNP id (column `SNPS`) from the GWAS catalog and write them to a file called `phenotype_SNP.txt`. Then create a file called `SNP_phenotype.txt` where the SNP column comes first, and the Disease/trait one second (`cut` alone is not enough for this, you need to create 2 files and paste them together).
</div>

<button type="button" class="btn btn-primary" onClick="code_toggle()">Click here to show/hide answers</button>



In [9]:
%%bash

cut -f8,22 GWAS_catalog_2017.05.12 > phenotype_SNP.txt

cut -f8 GWAS_catalog_2017.05.12 > phenotype.txt
cut -f22 GWAS_catalog_2017.05.12 > SNP.txt

paste SNP.txt phenotype.txt > SNP_phenotype.txt

## Step 5 : Find and replace

Once we have extracted some data, we often want to modify it. There are 2 programs to do this type of thing, one very simple and the other very complicated:

* `tr` replaces every occurrence of one particular character by another character
*  * The syntax is very simple : `command1 | tr 'a' 'b' ` replaces every occurrence of the letter a in the pipe by the letter b.
* `sed` is more complicated to use, but allows to substitute pretty much anything by pretty much anything else.
*  * `command1 | sed 's/pattern_to_find/replacement pattern/'`
*  * `pattern_to_find` is a **regular expression**, a special command that allow you to match certain bits of text an not others. Regular expressions are very, very common in the computing world, and many programs understand them (`grep` also allows them). Regular expressions (or regexes) can take a long time to learn and use properly. You can learn about them [here](http://www.regular-expressions.info/) and test them out [here](http://regexr.com/).

### Some (very) basic regular expressions

* `/Volos/` matches the string `Volos`, as expected
* `/Vo*los/` matches Volos with zero or more first `o`: `Volos`, `Vlos` `Voooooooolos`
* `/Vo+los/` matches Volos with one or more first `o`: `Volos`,  `Voooooooolos` but not `Vlos`
* `/V.los/` matches Volos, but the second letter can be anything (but not nothing): `Volos`, `Vylos`, `Vvlos` but not `Voolos`
* `/$/` matches the end of the line
* `/^/` matches the beginning of the line
* `/^Volos$/` matches a line that contains only `Volos` but not if there is a space (or anything) before or after.
* `s/o/i/` will replace `Volos` by `Vilos`, but `s/o/i/g` will replace all occurrences of the pattern: `Volos` will become `Vilis`.
* etc...


<br><br><br>

<div class="alert alert-success"><b>Question 8:</b> Using `tr`, replace every space character in the GWAS catalog by an underscore (`_`) and write it to `GWAS_catalog_no_spaces.txt`. Remember they are special characters, so need to be quoted.
</div>

<div class="alert alert-success"><b>Question 9:</b> In the file `phenotype_SNP.txt` you generated before, some SNPs have several IDs separated by a semicolon. Find them and remove all alternate IDs, keeping only the first one; write the file to `duplicates_removed.txt`.
</div>

<button type="button" class="btn btn-primary" onClick="code_toggle()">Click here to show/hide answers</button>


In [ ]:
%%bash
## Question 8
cat GWAS_catalog_2017.05.12 | tr ' ' '_' > GWAS_catalog_no_spaces.txt

## Question 9
grep ';' phenotype_SNP.txt | sed 's/;.*//' > duplicates_removed.txt

### Step 6 : Advanced column manipulation using `awk`


In [ ]:
%%bash

# extracting variants that are associated with height:
grep height filename
grep -i height filename
grep 


# extracting lines that are associated with 

## Variables

Temporarily storing a piece of information that we are going to refer later using its name. Two basic actions:
1. Setting the value (assignment).
2. Reading the value (The shell substitues the name of the variable with its value).

In [10]:
%%bash

# Setting the value a local variable (no spaces around the = sign):
chromosome=12 

# Calling variable:
echo 1. $chromosome
echo 2. ${chromosome}
echo 3. "${chromosome}"

# When curly braces are needed:
ls gene_list_chr$chromosome_lst # Warning message! The variable name is not separated from the rest of the string!
ls gene_list_chr${chromosome}_lst

1. 12
2. 12
3. 12
gene_list_chr12_lst


ls: gene_list_chr: No such file or directory


Often automatic value assignment is required when the value is read derived from an other process. Imagine a list of files

In [11]:
%%bash



UsageError: %%bash is a cell magic, but the cell body is empty.


## Loops

Often we have to repeat a set of steps multiple times. In such cases loops are constructed to keep the code short and clean. In bash we mainly use the `for` and the `while` loops. 

In [17]:
%%bash

# Repeating a process for all autosomes:
for chr in {1..22}; do
    echo Processing ${chr}
    # Some commands... 
done

Processing 1
Processing 2
Processing 3
Processing 4
Processing 5
Processing 6
Processing 7
Processing 8
Processing 9
Processing 10
Processing 11
Processing 12
Processing 13
Processing 14
Processing 15
Processing 16
Processing 17
Processing 18
Processing 19
Processing 20
Processing 21
Processing 22


In the above example using the curly braces, we generate a series of numbers between 1 and 22. Then in each this value is assigned to variable named `chr` in 